In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/univ/4/nlp/gpt

Mounted at /content/drive
/content/drive/My Drive/univ/4/nlp/gpt


In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
import pandas as pd

# OpenAI API 키 설정
api_key = 'YOUR_API'
client = OpenAI(api_key=api_key)

def get_gpt3_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # GPT-3.5 모델 엔진 이름
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,  # 원하는 응답 길이
        n=1,  # 하나의 응답 생성
        stop=None,
        temperature=0.7
    )
    return response.choices[0].message.content

# CSV 파일에서 프롬프트 읽기
csv_file = 'test_data_with_query.csv'
df = pd.read_csv(csv_file)

# 응답 저장을 위한 리스트 초기화
responses = []

# 각 프롬프트에 대해 GPT-3.5 응답 받기
for index, row in df.iterrows():
    prompt = '다음은 내가 처한 상황이야. 변호사로서 내 상황에 적용할 수 있는 법령과 그 이유를 설명해줘.'+row['자연어 쿼리']
    response = get_gpt3_response(prompt)
    responses.append(response)

# 응답을 새로운 열로 데이터프레임에 추가
df['response'] = responses

# 결과를 새로운 CSV 파일로 저장
output_csv_file = 'baseline1_gpt_result.csv'
df.to_csv(output_csv_file, index=False)

print(f'Responses saved to {output_csv_file}')

In [ ]:
import pandas as pd
import numpy as np
from metrics import exact_match_score, accuracy, f1

csv_file = 'baseline1_gpt_result.csv'
df = pd.read_csv(csv_file)

#예측값과 실제 정답 추출
preds = df['response'].tolist()
ground_truths = df['법령의 적용'].tolist()

# F1 점수 계산
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")

F1 Score: 4.096846444005376%


In [4]:
!pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9f919c624fd637855559f87b3c4dabc11edd1390efbe35499852ee8c68911525
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")

# BLEU 점수 계산
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")
# ROUGE 점수 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU Score: 0.011073084451736987%
BLEU-1 Score: 2.666809496958117%
BLEU-2 Score: 0.9906743310973829%
ROUGE-1 Score: 17.89867111628371%
ROUGE-2 Score: 4.202293570122077%
ROUGE-L Score: 16.02614829867896%


In [7]:
#baseline1 gpt inter
import pandas as pd
import numpy as np
from metrics import exact_match_score, accuracy, f1
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# CSV 파일 읽기
alpha_csv_file = 'AlphaMist_total_result.csv'
alpha_df = pd.read_csv(alpha_csv_file)
csv_file = 'baseline2_gpt_result.csv'
df = pd.read_csv(csv_file)
alpha3_csv_file='baseline3_AlphaMist_result.csv'
alpha3_df=pd.read_csv(alpha3_csv_file)


# 유효한 인덱스 필터링
valid_indices = alpha_df[alpha_df['답변'].notna() & alpha_df['RAG  답변'].notna()&alpha3_df['response'].notna()].index

csv_file2 = 'baseline1_gpt_result.csv'

df2 = pd.read_csv(csv_file2)

# 예측값과 실제값 추출
preds = df2.loc[valid_indices, 'response'].tolist()
ground_truths = df2.loc[valid_indices, '법령의 적용'].tolist()

# F1 점수 계산
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")
bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")
# BLEU 점수 계산
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")

# ROUGE 점수 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")

F1 Score: 3.9873382619116113%
BLEU Score: 1.576354856250077e-78%
BLEU-1 Score: 2.705484089403706%
BLEU-2 Score: 0.9905931952945347%
ROUGE-1 Score: 18.43971422791452%
ROUGE-2 Score: 4.544128196028616%
ROUGE-L Score: 16.470634399212873%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [6]:
print(f"Number of valid indices: {len(valid_indices)}")

Number of valid indices: 62
